<a href="https://colab.research.google.com/github/Saurabh703/Deep-Learning/blob/main/face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input 
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 

In [5]:
IMAGE_SIZE = [224,224]

In [6]:
train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

In [7]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [9]:
# useful for getting number of classes
folders = glob('Datasets/Train/*')

In [10]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

# x = Dense(len(folders), activation = 'relu')(x) 
prediction = Dense(len(folders),activation='softmax')(x)

In [11]:
# create model object 
model = Model(inputs=vgg.input, outputs=prediction)

In [12]:
# view the structure of the model 
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
model.compile(
    loss='categorical_classentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [14]:
from keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

FileNotFoundError: ignored

In [ ]:
# fit the model
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs = 5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
    )

In [ ]:
# loss 
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# Accuracies

plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label = 'val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('facefeatures_new_model.h5')